<a href="https://colab.research.google.com/github/khairihr/MachineLearning/blob/main/TaskWeek13/VGGNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama: Khairi Hibatullah Ridho

NIM: 1103228240

https://poloclub.github.io/cnn-explainer/

https://convnetplayground.fastforwardlabs.com/#/

LeNet-5, AlexNet and VGGNet.

https://landing.ai/ https://www.ultralytics.com/

##VGGNet

VGGNet (Visual Geometry Group Network) adalah salah satu arsitektur jaringan saraf tiruan konvolusi (CNN) yang sangat terkenal dalam pengenalan gambar dan komputer visi. Arsitektur ini dikembangkan oleh tim peneliti dari Visual Geometry Group di Universitas Oxford pada tahun 2014. VGGNet dikenal karena kekonsistensiannya dalam desain, dengan struktur yang dalam dan mendalam, yang menjadi referensi dalam pengembangan arsitektur CNN yang lebih kompleks.

Berikut adalah beberapa karakteristik utama dari arsitektur VGGNet:

1. **Kedalaman yang Dalam**: Salah satu ciri utama VGGNet adalah kedalaman arsitekturnya. Arsitektur ini terdiri dari beberapa lapisan konvolusi (Convolutional layers) yang dalam. Terdapat dua varian utama dari VGGNet: VGG16, yang memiliki 16 lapisan (13 lapisan konvolusi dan 3 lapisan Fully Connected), dan VGG19, yang memiliki 19 lapisan (16 lapisan konvolusi dan 3 lapisan Fully Connected). Kedalaman ini membantu dalam ekstraksi fitur hierarkis yang lebih kompleks dari gambar.

2. **Ukuran Kernel Konstan**: VGGNet menggunakan kernel konvolusi berukuran kecil (biasanya 3x3 piksel) dengan langkah (stride) 1 dan padding 1 untuk semua lapisan konvolusi. Hal ini menghasilkan operasi konvolusi yang kecil dan seragam, yang membantu dalam pemahaman fitur pada berbagai skala.

3. **Max-Pooling**: Setelah beberapa lapisan konvolusi, VGGNet menggunakan max-pooling dengan ukuran filter 2x2 piksel dan langkah 2x2 piksel untuk mengurangi dimensi gambar. Ini membantu dalam mengurangi overfitting dan menghasilkan representasi fitur yang lebih kecil.

4. **Fully Connected Layers**: Setelah lapisan-lapisan konvolusi dan max-pooling, VGGNet memiliki beberapa lapisan Fully Connected (FC) yang diikuti oleh fungsi aktivasi ReLU. Lapisan FC ini digunakan untuk menggabungkan fitur-fitur yang telah diekstraksi sebelumnya untuk klasifikasi.

5. **Fungsi Aktivasi ReLU**: ReLU (Rectified Linear Unit) digunakan sebagai fungsi aktivasi di seluruh arsitektur VGGNet. Ini memberikan non-linearitas ke jaringan dan membantu mengatasi masalah vanishng gradient yang umum terjadi dalam jaringan yang dalam.

6. **Klasifikasi Multi-Kelas**: VGGNet biasanya digunakan untuk tugas klasifikasi multi-kelas, seperti mengenali objek dalam gambar pada dataset ImageNet, di mana terdapat ribuan kelas berbeda.

Arsitektur VGGNet telah menjadi dasar bagi pengembangan banyak arsitektur CNN yang lebih kompleks, termasuk ResNet, Inception, dan lainnya. Meskipun VGGNet memiliki banyak parameter, ia terkenal karena kemampuannya dalam menghasilkan representasi fitur yang kuat, meskipun dengan biaya komputasi yang tinggi.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the VGGNet architecture for MNIST
class VGGNet(nn.Module):
    def __init__(self):
        super(VGGNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(256 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 10),  # 10 classes for MNIST
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Load MNIST dataset
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Initialize model, loss function, and optimizer
model = VGGNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

# Testing
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predictions = torch.max(outputs, 1)

        total_correct += (predictions == labels).sum().item()
        total_samples += labels.size(0)

accuracy = total_correct / total_samples
print(f"Test Accuracy: {accuracy}")

100%|██████████| 9912422/9912422 [00:00<00:00, 156962923.20it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 38763422.02it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 36940115.03it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 13415865.33it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch 1/5, Loss: 0.21509855238299794
Epoch 2/5, Loss: 0.06588682752374074
Epoch 3/5, Loss: 0.05543005707881799
Epoch 4/5, Loss: 0.04790577211272731
Epoch 5/5, Loss: 0.040858554438265374
Test Accuracy: 0.9883


 Berikut adalah penjelasan langkah demi langkah dari kode tersebut:

1. **Impor Library**: Kode dimulai dengan mengimpor beberapa pustaka PyTorch yang diperlukan, termasuk `torch`, `torch.nn`, `torch.optim`, `torchvision`, `transforms`, dan `DataLoader`. Ini termasuk pengaturan perangkat (CPU atau GPU) yang akan digunakan untuk pelatihan.

2. **Definisi Arsitektur VGGNet**: Anda mendefinisikan arsitektur VGGNet dalam kelas `VGGNet`. Arsitektur ini mencakup lapisan-lapisan konvolusi, ReLU, max-pooling, dan lapisan-lapisan linear untuk klasifikasi. Anda menggunakan `nn.Sequential` untuk mengelompokkan lapisan-lapisan konvolusi dan aktivasi.

3. **Transformasi Data**: Anda mendefinisikan transformasi data untuk dataset MNIST. Ini termasuk mengubah gambar menjadi tensor dan menyesuaikan ukuran gambar menjadi 28x28 piksel.

4. **Memuat Dataset**: Anda memuat dataset MNIST menggunakan `torchvision.datasets.MNIST`. Ini akan mengunduh dataset jika belum ada di direktori yang ditentukan. Selanjutnya, Anda membuat dua objek `DataLoader` untuk data pelatihan dan pengujian. DataLoader digunakan untuk memecah data menjadi batch-batch yang dapat digunakan untuk pelatihan dan pengujian model.

5. **Inisialisasi Model, Fungsi Loss, dan Optimizer**: Anda membuat objek model `VGGNet`, memilih fungsi loss `nn.CrossEntropyLoss()` untuk tugas klasifikasi, dan memilih optimizer `optim.Adam` dengan learning rate 0.001.

6. **Pelatihan**: Anda melakukan pelatihan model menggunakan loop `for` yang berjalan sebanyak `num_epochs`. Dalam loop pelatihan, Anda mengirimkan batch data ke model, menghitung loss, melakukan backpropagation, dan memperbarui bobot model melalui optimizer. Setiap epoch mencatat loss rata-rata selama iterasi pelatihan dan mencetaknya.

7. **Pengujian**: Setelah pelatihan selesai, Anda melakukan pengujian model pada dataset pengujian. Model dinilai pada mode evaluasi dengan menggunakan `model.eval()`. Anda menghitung jumlah prediksi yang benar dan total sampel untuk mengukur akurasi pengujian.

8. **Akurasi Pengujian**: Akurasi pengujian dihitung dengan membagi jumlah prediksi yang benar dengan total sampel dan dicetak.

Kode ini akan melatih model VGGNet untuk mengenali digit dalam dataset MNIST dan mencetak akurasi pengujian setelah pelatihan selesai. Meskipun VGGNet biasanya digunakan untuk dataset gambar dengan resolusi lebih tinggi, Anda telah mengadaptasinya untuk dataset MNIST dengan mengubah arsitektur dan ukuran gambar.